In [1]:
import pandas as pd

df = pd.read_parquet('hackathon_files_for_participants_ozon/train_data.parquet')

In [2]:
df.head()

,variantid,name,categories,color_parsed,pic_embeddings_resnet_v1,main_pic_embeddings_resnet_v1,name_bert_64,characteristic_attributes_mapping
0,51195767,"Удлинитель Партнер-Электро ПВС 2х0,75 ГОСТ,6A,...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[оранжевый],None,"[[0.04603629, 0.18839523, -0.09973055, -0.6636...","[-0.47045058, 0.67237014, 0.48984158, -0.54485...","{""Номинальный ток, А"":[""10""],""Цвет товара"":[""о..."
1,53565809,Магнитный кабель USB 2.0 A (m) - USB Type-C (m...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Кабели ...",[красный],"[[0.26863545, -0.3130674, 0.29023397, 0.073978...","[[1.1471839, -0.665361, 0.7745614, 0.26716197,...","[-0.6575592, 0.6522429, 0.5426037, -0.54347897...","{""Конструктивные особенности"":[""Магнитная конс..."
2,56763357,"Набор микропрепаратов Konus 25: ""Клетки и ткан...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Оптичес...",None,"[[0.66954195, 1.0643557, 0.78324044, -0.338267...","[[-0.90570974, 1.0296293, 1.0769907, 0.27746, ...","[-0.7384308, 0.70784587, 0.3012653, -0.3583719...","{""Тип аксессуара"":[""Набор микропрепаратов""],""Б..."
3,56961772,"Мобильный телефон BQ 1848 Step, черный","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Смартфо...",[черный],"[[0.6580482, -0.35763323, -0.16939065, -0.4249...","[[0.13133773, -0.5577079, 0.32498044, 0.191717...","[-0.44812852, 0.5283565, 0.28981736, -0.506841...","{""Тип карты памяти"":[""microSD""],""Число SIM-кар..."
4,61054740,"Штатив трипод Tripod 330A для фотоаппаратов, в...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Штативы...",[черный],"[[-0.10406649, 0.080646515, -0.28668788, 0.739...","[[0.21696381, 0.10989461, -0.08012986, 0.69186...","[-0.72692573, 0.75206333, 0.37740713, -0.52502...","{""Материал"":[""Металл""],""Количество секций, шт""..."


In [3]:
df_train_pairs = pd.read_parquet('hackathon_files_for_participants_ozon/train_pairs.parquet')

In [4]:
df['cat3'] = df['categories'].apply(lambda x : eval(x)['3'])

In [5]:
df['cat3']

0         Сетевые фильтры, разветвители и удлинители
1                               Кабели и переходники
2                                 Оптические приборы
3            Смартфоны, планшеты, мобильные телефоны
4                                  Штативы и головки
                             ...                    
457058                          Расходник для печати
457059                      Защитные пленки и стекла
457060                                     Компьютер
457061                                     Компьютер
457062       Смартфоны, планшеты, мобильные телефоны
Name: cat3, Length: 457063, dtype: object

In [6]:
cat3_counts = df.cat3.value_counts().to_dict()
df["cat3_groupped"] = df["cat3"].apply(lambda x: x if cat3_counts[x] > 1000 else "rest")

In [7]:
import tqdm
def making_char_pairs(js1, js2):
    res_dist, res_similar = [], []
    try:
        js1 = eval(js1)
        js2 = eval(js2)
        jskeys = set(js1.keys()) & set(js2.keys())
    except:
        return res_dist, res_similar
    
    for k in jskeys:
        v1 = js1.get(k)
        v2 = js2.get(k)
        if v1 != v2:
            res_dist.append(k)
        if v1 == v2:
            res_similar.append(k)
    return res_dist, res_similar

dataset = []
for i in tqdm.tqdm(range(len(df_train_pairs))):
    t1,t2 = df_train_pairs.iloc[i].variantid1, df_train_pairs.iloc[i].variantid2
    target = df_train_pairs.iloc[i].target
    category = df.loc[df.variantid == t1].cat3.values[0]
    name1 = df.loc[df.variantid == t1].name.values[0]
    name2 = df.loc[df.variantid == t2].name.values[0]
    cat_groupped = df.loc[df.variantid == t1].cat3_groupped.values[0]
    res_dist, res_similar = making_char_pairs(df.loc[df.variantid == t1].characteristic_attributes_mapping.values[0],
                                              df.loc[df.variantid == t2].characteristic_attributes_mapping.values[0]
                                             )
    dataset.append((category, name1, name2, ', '.join(res_dist), ', '.join(res_similar), target, cat_groupped))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 306540/306540 [20:53<00:00, 244.51it/s]


In [8]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve, auc


def pr_auc_macro(
    df: pd.DataFrame,
    prec_level: float = 0.75
) -> float:
        
    y_true = df["target"]
    y_pred = df["scores"]
    categories = df["categories"]
    
    weights = []
    pr_aucs = []

    unique_cats, counts = np.unique(categories, return_counts=True)

    # calculate metric for each big category
    for i, category in enumerate(unique_cats):
        # take just a certain category
        cat_idx = np.where(categories == category)[0]
        y_pred_cat = y_pred[cat_idx]
        y_true_cat = y_true[cat_idx]

        # if there is no matches in the category then PRAUC=0
        if sum(y_true_cat) == 0:
            pr_aucs.append(0)
            weights.append(counts[i] / len(categories))
            continue
        
        # get coordinates (x, y) for (recall, precision) of PR-curve
        y, x, _ = precision_recall_curve(y_true_cat, y_pred_cat)
        
        # reverse the lists so that x's are in ascending order (left to right)
        y = y[::-1]
        x = x[::-1]
        
        # get indices for x-coordinate (recall) where y-coordinate (precision) 
        # is higher than precision level (75% for our task)
        good_idx = np.where(y >= prec_level)[0]
        
        # if there are more than one such x's (at least one is always there, 
        # it's x=0 (recall=0)) we get a grid from x=0, to the rightest x 
        # with acceptable precision
        if len(good_idx) > 1:
            gt_prec_level_idx = np.arange(0, good_idx[-1] + 1)
        # if there is only one such x, then we have zeros in the top scores 
        # and the curve simply goes down sharply at x=0 and does not rise 
        # above the required precision: PRAUC=0
        else:
            pr_aucs.append(0)
            weights.append(counts[i] / len(categories))
            continue
        
        # calculate category weight anyway
        weights.append(counts[i] / len(categories))
        # calculate PRAUC for all points where the rightest x 
        # still has required precision 
        try:
            pr_auc_prec_level = auc(x[gt_prec_level_idx], y[gt_prec_level_idx])
            if not np.isnan(pr_auc_prec_level):
                pr_aucs.append(pr_auc_prec_level)
        except ValueError:
            pr_aucs.append(0)
            
    return np.average(pr_aucs, weights=weights)

In [9]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import tqdm
from transformers import get_linear_schedule_with_warmup

kf = KFold(n_splits=5, shuffle=True, random_state=239)
ds_indexes = np.arange(len(dataset))

ifold = 0
oof = np.zeros(len(dataset))
for tr, va in kf.split(ds_indexes):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
    model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", 
                                                               num_labels=2).cuda()
    #model.load_state_dict(torch.load(models_by_folds[ifold]))
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
    epochs = 1
    total_steps = (1 + len(tr) // 32) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, 
                                                num_training_steps = total_steps)
    
    i = 0
    train_losses = []
    for ep in range(epochs):
        np.random.shuffle(tr)
        optimizer.zero_grad()
        losses = []
        for t in tr:
            category, name1, name2, dist, sim, target, _ = dataset[t]
            s = category + '[SEP]' + name1 + '[SEP]' + name2 + '[SEP]' + dist # + '[SEP]' + sim
            tks = tokenizer.encode_plus(s[:1200], max_length=512, pad_to_max_length=False, 
                                        return_attention_mask=True, return_tensors='pt', truncation=True)
            out = model(tks['input_ids'].cuda(), 
                        attention_mask=tks['attention_mask'].cuda(),
                        token_type_ids=tks['token_type_ids'].cuda(),
                        labels = torch.tensor([[1.0-target, target]]).float().cuda()
                       )
            #print(out.logits[0][1].item(), target)
            losses.append(out.loss)
            
            i += 1
            if i % 32 == 0:
                loss = sum(losses) / 32.0
                loss.backward()
                losses = []
                train_losses.append(loss.item())
                optimizer.step() 
                optimizer.zero_grad()
                scheduler.step()
                
        if len(losses) > 0:
            loss = sum(losses) / 32.0
            loss.backward()
            losses = []
            train_losses.append(loss.item())
            optimizer.step() 
            optimizer.zero_grad()
            scheduler.step()
            
        evaldf = []
        for t in va:
            category, name1, name2, dist, sim, target, cat_groupped = dataset[t]
            s = category + '[SEP]' + name1 + '[SEP]' + name2 + '[SEP]'  + dist #+ '[SEP]' + sim
            tks = tokenizer.encode_plus(s[:1200], max_length=512, pad_to_max_length=False,
                                return_attention_mask=True, return_tensors='pt', truncation=True)

            with torch.no_grad():
                score = model(tks['input_ids'].cuda(), 
                              attention_mask=tks['attention_mask'].cuda(),
                              token_type_ids=tks['token_type_ids'].cuda()).logits[0][1].item()
            evaldf.append((target, score, cat_groupped))
        evaldf = pd.DataFrame(evaldf)
        evaldf.columns = ["target", "scores", "categories"]
        m = pr_auc_macro(evaldf)
        m2 = roc_auc_score(evaldf.target.values, evaldf.scores.values)
        print('fold', ifold, 'epoch', ep, 'prauc(0.75)', round(m,3), 'rocauc', round(m2,3))
        torch.save(model.state_dict(), f'chstic_{ifold}_{round(m,3)}_{round(m2,3)}.pth')
        oof[va] = evaldf.scores.values
    ifold += 1
    
df_oof = pd.read_parquet('hackathon_files_for_participants_ozon/train_pairs.parquet')
df_oof['chstic'] = oof
df_oof.to_parquet('oof_chstic.parquet')

nn_features = pd.read_parquet('oof_mbert.parquet')["variantid1","variantid2","mbert"]
nn_features = nn_features.merge(evaldf, on=["variantid1","variantid2"], how='left')
nn_features.to_parquet('nn/train.parquet')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

fold 0 epoch 0 prauc(0.75) 0.762 rocauc 0.937


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

fold 1 epoch 0 prauc(0.75) 0.767 rocauc 0.937


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

fold 2 epoch 0 prauc(0.75) 0.764 rocauc 0.937


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

fold 3 epoch 0 prauc(0.75) 0.762 rocauc 0.937


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

fold 4 epoch 0 prauc(0.75) 0.763 rocauc 0.936
